<B>TCC PUC MINAS - MAIO/2022</B><BR>
ANÁLISE DE PARCELAMENTO DE DÉBITOS INSCRITOS EM DÍVIDA ATIVA DA UNIÃO ATRAVÉS DE MACHINE LEARNING<BR>
FELIPE MARTINS<BR>

05 - DÉBITOS PGFN
    
IMPORTAÇÃO E TRATAMENTO DO DATASET DO ESTOQUE DE DÉBITOS INSCRITOS EM DAU EM 1TRIM/2022
    
Tempo médio de processamento completo deste notebook: 10 min

In [1]:
import pandas as pd

## DATASET DE DEBITOS DA PGFN DO 1º TRIMESTRE/2022 (df) ##

In [3]:
%%time
# IMPORTANDO DATASET DE DEBITOS DA PGFN DO 4º TRIMESTRE/2020
# Importando dataset de uma pasta compactada com vários arquivos e concatenando-os
# https://samukweku.github.io/data-wrangling-blog/python/pandas/compressed%20data/zip/archived%20data/2020/07/21/Extract-DataFrame-from-Compressed-Data-into-Pandas.html

from zipfile import ZipFile

with ZipFile("pgfn\Dados_abertos_Nao_Previdenciario_1trim2022.zip") as zipfiles:
    #the first entry is the zipfile name
    #we'll skip it
    filelist = zipfiles.namelist()[1:]
    
    data = [pd.read_csv(zipfiles.open(file_name), usecols=[0, 1, 2, 6, 7, 8, 9, 10, 11, 12], dtype={"NUMERO_INSCRICAO":"category"}, sep= ';', encoding='unicode_escape') for file_name in filelist]  #nrows=5

df = pd.concat(data)

df.head()

Wall time: 2min 1s


,CPF_CNPJ,TIPO_PESSOA,TIPO_DEVEDOR,NUMERO_INSCRICAO,TIPO_SITUACAO_INSCRICAO,SITUACAO_INSCRICAO,RECEITA_PRINCIPAL,DATA_INSCRICAO,INDICADOR_AJUIZADO,VALOR_CONSOLIDADO
0,XXX099.702XX,Pessoa física,CORRESPONSAVEL,2142200065230,Em cobrança,ATIVA EM COBRANCA,R D ATIVA - SIMP NAC - MEI,24/01/2022,NAO,3122.93
1,11.859.590/0001-29,Pessoa jurídica,PRINCIPAL,2141900331458,Em cobrança,ATIVA EM COBRANCA,DIV ATIVA-SIMPLES NACIONAL,18/06/2019,NAO,55464.40
2,05.574.789/0001-08,Pessoa jurídica,PRINCIPAL,2151000016030,Em cobrança,ATIVA AJUIZADA,Receita da dívida ativa - Multa - CLT,27/01/2010,SIM,55446.40
3,XXX856.122XX,Pessoa física,PRINCIPAL,2142000534105,Benefício Fiscal,ATIVA NAO AJUIZAVEL NEGOCIADA NO SISPAR,R D Ativa - Contribuição Previdenciária Segurados,26/10/2020,NAO,3182.70
4,14.177.984/0001-95,Pessoa jurídica,PRINCIPAL,2160000092445,Em cobrança,ATIVA AJUIZADA,Receita da dívida ativa - COFINS,27/09/2000,SIM,3457828.22


In [4]:
# RESETANDO O INDEX
# MOTIVO: AO IMPORTAR VÁRIOS ARQUIVOS AO MESMO TEMPO, O INDICE REINICIA DO ZERO PARA CADA ARQUIVO.
# POR EXEMPLO: COMO SÃO 26 ARQUIVOS, O DATASET FICA COM 26 LINHAS DE INDICES IGUAIS (26 LINHAS INDICE 0, 26 LINHAS INDICE 1, ...)
# ISSO COMPROMETE TODA A MANIPULAÇÃO DO DS, POIS AO SE TENTAR ALTERAR UMA LINHA ESPECÍFICA, TODAS AS LINHAS DE MESMO INDICE SÃO ALTERADAS.
# O COMANDO ABAIXO RESETA O INDICE E RESOLVE O PROBLEMA. O PARÂMETRO drop=True É PARA NÃO CRIAR UMA NOVA COLUNA COM ÍNDICE.
df = df.reset_index(drop=True)

In [5]:
df.head()

,CPF_CNPJ,TIPO_PESSOA,TIPO_DEVEDOR,NUMERO_INSCRICAO,TIPO_SITUACAO_INSCRICAO,SITUACAO_INSCRICAO,RECEITA_PRINCIPAL,DATA_INSCRICAO,INDICADOR_AJUIZADO,VALOR_CONSOLIDADO
0,XXX099.702XX,Pessoa física,CORRESPONSAVEL,2142200065230,Em cobrança,ATIVA EM COBRANCA,R D ATIVA - SIMP NAC - MEI,24/01/2022,NAO,3122.93
1,11.859.590/0001-29,Pessoa jurídica,PRINCIPAL,2141900331458,Em cobrança,ATIVA EM COBRANCA,DIV ATIVA-SIMPLES NACIONAL,18/06/2019,NAO,55464.40
2,05.574.789/0001-08,Pessoa jurídica,PRINCIPAL,2151000016030,Em cobrança,ATIVA AJUIZADA,Receita da dívida ativa - Multa - CLT,27/01/2010,SIM,55446.40
3,XXX856.122XX,Pessoa física,PRINCIPAL,2142000534105,Benefício Fiscal,ATIVA NAO AJUIZAVEL NEGOCIADA NO SISPAR,R D Ativa - Contribuição Previdenciária Segurados,26/10/2020,NAO,3182.70
4,14.177.984/0001-95,Pessoa jurídica,PRINCIPAL,2160000092445,Em cobrança,ATIVA AJUIZADA,Receita da dívida ativa - COFINS,27/09/2000,SIM,3457828.22


In [6]:
df.count()

CPF_CNPJ                   19651579
TIPO_PESSOA                19651579
TIPO_DEVEDOR               19651579
NUMERO_INSCRICAO           19651579
TIPO_SITUACAO_INSCRICAO    19651579
SITUACAO_INSCRICAO         19651579
RECEITA_PRINCIPAL          19651579
DATA_INSCRICAO             19651579
INDICADOR_AJUIZADO         19651579
VALOR_CONSOLIDADO          19651579
dtype: int64

In [7]:
df.dtypes

CPF_CNPJ                    object
TIPO_PESSOA                 object
TIPO_DEVEDOR                object
NUMERO_INSCRICAO            object
TIPO_SITUACAO_INSCRICAO     object
SITUACAO_INSCRICAO          object
RECEITA_PRINCIPAL           object
DATA_INSCRICAO              object
INDICADOR_AJUIZADO          object
VALOR_CONSOLIDADO          float64
dtype: object

In [8]:
%%time
# CONVERTENDO CAMPO DATA_INSCRICAO PARA TIPO DATETIME
df["DATA_INSCRICAO"] = df["DATA_INSCRICAO"].astype("datetime64")

Wall time: 3.22 s


In [9]:
df.dtypes

CPF_CNPJ                           object
TIPO_PESSOA                        object
TIPO_DEVEDOR                       object
NUMERO_INSCRICAO                   object
TIPO_SITUACAO_INSCRICAO            object
SITUACAO_INSCRICAO                 object
RECEITA_PRINCIPAL                  object
DATA_INSCRICAO             datetime64[ns]
INDICADOR_AJUIZADO                 object
VALOR_CONSOLIDADO                 float64
dtype: object

In [10]:
df.head()

,CPF_CNPJ,TIPO_PESSOA,TIPO_DEVEDOR,NUMERO_INSCRICAO,TIPO_SITUACAO_INSCRICAO,SITUACAO_INSCRICAO,RECEITA_PRINCIPAL,DATA_INSCRICAO,INDICADOR_AJUIZADO,VALOR_CONSOLIDADO
0,XXX099.702XX,Pessoa física,CORRESPONSAVEL,2142200065230,Em cobrança,ATIVA EM COBRANCA,R D ATIVA - SIMP NAC - MEI,2022-01-24,NAO,3122.93
1,11.859.590/0001-29,Pessoa jurídica,PRINCIPAL,2141900331458,Em cobrança,ATIVA EM COBRANCA,DIV ATIVA-SIMPLES NACIONAL,2019-06-18,NAO,55464.40
2,05.574.789/0001-08,Pessoa jurídica,PRINCIPAL,2151000016030,Em cobrança,ATIVA AJUIZADA,Receita da dívida ativa - Multa - CLT,2010-01-27,SIM,55446.40
3,XXX856.122XX,Pessoa física,PRINCIPAL,2142000534105,Benefício Fiscal,ATIVA NAO AJUIZAVEL NEGOCIADA NO SISPAR,R D Ativa - Contribuição Previdenciária Segurados,2020-10-26,NAO,3182.70
4,14.177.984/0001-95,Pessoa jurídica,PRINCIPAL,2160000092445,Em cobrança,ATIVA AJUIZADA,Receita da dívida ativa - COFINS,2000-09-27,SIM,3457828.22


In [11]:
df.isnull().sum()

CPF_CNPJ                   0
TIPO_PESSOA                0
TIPO_DEVEDOR               0
NUMERO_INSCRICAO           0
TIPO_SITUACAO_INSCRICAO    0
SITUACAO_INSCRICAO         0
RECEITA_PRINCIPAL          0
DATA_INSCRICAO             0
INDICADOR_AJUIZADO         0
VALOR_CONSOLIDADO          0
dtype: int64

## Ajustando campo TIPO_DEVEDOR
Removendo-se o tipo de devedor diferente de principal

In [12]:
print(df["TIPO_DEVEDOR"].value_counts())

PRINCIPAL         14992922
CORRESPONSAVEL     4642696
SOLIDARIO            15961
Name: TIPO_DEVEDOR, dtype: int64


In [13]:
%%time
# REMOVENDO TODOS AS LINHAS DA COLUNA 'TIPO_DEVEDOR' QUE SÃO DIFERENTES DE 'PRINCIPAL' (OU SEJA, REMOVENDO SOLIDÁRIOS E CORRESPONSÁVEL PARA EVITAR DUPLICIDADE DE INSCRIÇÃO)
# PROCESSAMENTO DEMORADO (cerca de 10min)
df = df.drop(df.loc[df['TIPO_DEVEDOR']!='PRINCIPAL'].index)

Wall time: 7.76 s


In [14]:
print(df["TIPO_DEVEDOR"].value_counts())

PRINCIPAL    14992922
Name: TIPO_DEVEDOR, dtype: int64


In [15]:
df.count()

CPF_CNPJ                   14992922
TIPO_PESSOA                14992922
TIPO_DEVEDOR               14992922
NUMERO_INSCRICAO           14992922
TIPO_SITUACAO_INSCRICAO    14992922
SITUACAO_INSCRICAO         14992922
RECEITA_PRINCIPAL          14992922
DATA_INSCRICAO             14992922
INDICADOR_AJUIZADO         14992922
VALOR_CONSOLIDADO          14992922
dtype: int64

In [16]:
df.head()

,CPF_CNPJ,TIPO_PESSOA,TIPO_DEVEDOR,NUMERO_INSCRICAO,TIPO_SITUACAO_INSCRICAO,SITUACAO_INSCRICAO,RECEITA_PRINCIPAL,DATA_INSCRICAO,INDICADOR_AJUIZADO,VALOR_CONSOLIDADO
1,11.859.590/0001-29,Pessoa jurídica,PRINCIPAL,2141900331458,Em cobrança,ATIVA EM COBRANCA,DIV ATIVA-SIMPLES NACIONAL,2019-06-18,NAO,55464.40
2,05.574.789/0001-08,Pessoa jurídica,PRINCIPAL,2151000016030,Em cobrança,ATIVA AJUIZADA,Receita da dívida ativa - Multa - CLT,2010-01-27,SIM,55446.40
3,XXX856.122XX,Pessoa física,PRINCIPAL,2142000534105,Benefício Fiscal,ATIVA NAO AJUIZAVEL NEGOCIADA NO SISPAR,R D Ativa - Contribuição Previdenciária Segurados,2020-10-26,NAO,3182.70
4,14.177.984/0001-95,Pessoa jurídica,PRINCIPAL,2160000092445,Em cobrança,ATIVA AJUIZADA,Receita da dívida ativa - COFINS,2000-09-27,SIM,3457828.22
5,02.793.722/0001-76,Pessoa jurídica,PRINCIPAL,2161900910100,Em cobrança,ATIVA EM COBRANCA,R D Ativa - Multa Isolada,2019-02-12,NAO,1521.30


## Ajustando campo TIPO_PESSOA
Substituindo-se os valores do campo ‘TIPO_PESSOA’ por PF e PJ

In [18]:
%%time
# SUBSTITUINDO O CAMPO TIPO_PESSOA POR PJ E PF
df['TIPO_PESSOA'] = df['TIPO_PESSOA'].replace(['Pessoa jurídica','Pessoa física'],['PJ','PF'])

Wall time: 1.37 s


In [19]:
print(df["TIPO_PESSOA"].value_counts())

PJ    12223696
PF     2769226
Name: TIPO_PESSOA, dtype: int64


In [20]:
df.head()

,CPF_CNPJ,TIPO_PESSOA,TIPO_DEVEDOR,NUMERO_INSCRICAO,TIPO_SITUACAO_INSCRICAO,SITUACAO_INSCRICAO,RECEITA_PRINCIPAL,DATA_INSCRICAO,INDICADOR_AJUIZADO,VALOR_CONSOLIDADO
1,11.859.590/0001-29,PJ,PRINCIPAL,2141900331458,Em cobrança,ATIVA EM COBRANCA,DIV ATIVA-SIMPLES NACIONAL,2019-06-18,NAO,55464.40
2,05.574.789/0001-08,PJ,PRINCIPAL,2151000016030,Em cobrança,ATIVA AJUIZADA,Receita da dívida ativa - Multa - CLT,2010-01-27,SIM,55446.40
3,XXX856.122XX,PF,PRINCIPAL,2142000534105,Benefício Fiscal,ATIVA NAO AJUIZAVEL NEGOCIADA NO SISPAR,R D Ativa - Contribuição Previdenciária Segurados,2020-10-26,NAO,3182.70
4,14.177.984/0001-95,PJ,PRINCIPAL,2160000092445,Em cobrança,ATIVA AJUIZADA,Receita da dívida ativa - COFINS,2000-09-27,SIM,3457828.22
5,02.793.722/0001-76,PJ,PRINCIPAL,2161900910100,Em cobrança,ATIVA EM COBRANCA,R D Ativa - Multa Isolada,2019-02-12,NAO,1521.30


In [21]:
df.count()

CPF_CNPJ                   14992922
TIPO_PESSOA                14992922
TIPO_DEVEDOR               14992922
NUMERO_INSCRICAO           14992922
TIPO_SITUACAO_INSCRICAO    14992922
SITUACAO_INSCRICAO         14992922
RECEITA_PRINCIPAL          14992922
DATA_INSCRICAO             14992922
INDICADOR_AJUIZADO         14992922
VALOR_CONSOLIDADO          14992922
dtype: int64

In [22]:
df.dtypes

CPF_CNPJ                           object
TIPO_PESSOA                        object
TIPO_DEVEDOR                       object
NUMERO_INSCRICAO                   object
TIPO_SITUACAO_INSCRICAO            object
SITUACAO_INSCRICAO                 object
RECEITA_PRINCIPAL                  object
DATA_INSCRICAO             datetime64[ns]
INDICADOR_AJUIZADO                 object
VALOR_CONSOLIDADO                 float64
dtype: object

## Criação do campo INSC_>1_ANO

In [23]:
%%time
# CRIANDO NOVO CAMPO CALCULADO DO TEMPO DE INSCRIÇÃO DA DÍVIDA (MAIOR QUE 1 ANO) UTILIZANDO DATA_INSCRICAO
# ATENÇÃO: PERÍODO-BASE É 1 TRIM/2022. ASSIM, SERÁ CONSIDERADA DÍVIDA MAIOR QUE 1 ANO AS INSCRITAS ATÉ 2019.
import numpy as np

df['ANO_INSCRICAO'] = df['DATA_INSCRICAO'].dt.year
df['INSC_>1_ANO'] = np.where(df.ANO_INSCRICAO<=2019, 'S', 'N')

Wall time: 1.39 s


In [24]:
df.head(100)

,CPF_CNPJ,TIPO_PESSOA,TIPO_DEVEDOR,NUMERO_INSCRICAO,TIPO_SITUACAO_INSCRICAO,SITUACAO_INSCRICAO,RECEITA_PRINCIPAL,DATA_INSCRICAO,INDICADOR_AJUIZADO,VALOR_CONSOLIDADO,ANO_INSCRICAO,INSC_>1_ANO
1,11.859.590/0001-29,PJ,PRINCIPAL,2141900331458,Em cobrança,ATIVA EM COBRANCA,DIV ATIVA-SIMPLES NACIONAL,2019-06-18,NAO,55464.40,2019,S
2,05.574.789/0001-08,PJ,PRINCIPAL,2151000016030,Em cobrança,ATIVA AJUIZADA,Receita da dívida ativa - Multa - CLT,2010-01-27,SIM,55446.40,2010,S
3,XXX856.122XX,PF,PRINCIPAL,2142000534105,Benefício Fiscal,ATIVA NAO AJUIZAVEL NEGOCIADA NO SISPAR,R D Ativa - Contribuição Previdenciária Segurados,2020-10-26,NAO,3182.70,2020,N
4,14.177.984/0001-95,PJ,PRINCIPAL,2160000092445,Em cobrança,ATIVA AJUIZADA,Receita da dívida ativa - COFINS,2000-09-27,SIM,3457828.22,2000,S
5,02.793.722/0001-76,PJ,PRINCIPAL,2161900910100,Em cobrança,ATIVA EM COBRANCA,R D Ativa - Multa Isolada,2019-02-12,NAO,1521.30,2019,S
...,...,...,...,...,...,...,...,...,...,...,...,...
115,15.070.453/0001-61,PJ,PRINCIPAL,2161500682703,Em cobrança,ATIVA AJUIZADA,Receita da dívida ativa - COFINS,2015-04-12,SIM,9853.33,2015,S
119,21.456.849/0001-44,PJ,PRINCIPAL,2162100036411,Em cobrança,ATIVA EM COBRANCA,Receita da dívida ativa - COFINS,2021-03-30,NAO,16976.04,2021,N
120,06.314.820/0001-34,PJ,PRINCIPAL,2121100100074,Em cobrança,ATIVA AJUIZADA,Receita da dívida ativa - IRPJ,2011-07-26,SIM,3551.05,2011,S
121,XXX520.602XX,PF,PRINCIPAL,2111900981108,Benefício Fiscal,ATIVA NAO AJUIZAVEL NEGOCIADA NO SISPAR,Receita da dívida ativa - IRPF,2019-08-27,NAO,8769.87,2019,S


## Criação do campo HA_OUTROS_DEBITOS

In [25]:
%%time
# CRIANDO CAMPO QUE VERIFICA SE HÁ MAIS DE UM DÉBITO (MAIS DE UMA LINHA PARA O MESMO CPF_CNPJ).

import numpy as np
df['CONTADOR_NOME'] = df.groupby('CPF_CNPJ')['CPF_CNPJ'].transform('count')
df['HA_OUTROS_DEBITOS'] = np.where(df.CONTADOR_NOME>1, 'S', 'N')

Wall time: 12.8 s


In [26]:
df.head()

,CPF_CNPJ,TIPO_PESSOA,TIPO_DEVEDOR,NUMERO_INSCRICAO,TIPO_SITUACAO_INSCRICAO,SITUACAO_INSCRICAO,RECEITA_PRINCIPAL,DATA_INSCRICAO,INDICADOR_AJUIZADO,VALOR_CONSOLIDADO,ANO_INSCRICAO,INSC_>1_ANO,CONTADOR_NOME,HA_OUTROS_DEBITOS
1,11.859.590/0001-29,PJ,PRINCIPAL,2141900331458,Em cobrança,ATIVA EM COBRANCA,DIV ATIVA-SIMPLES NACIONAL,2019-06-18,NAO,55464.40,2019,S,2,S
2,05.574.789/0001-08,PJ,PRINCIPAL,2151000016030,Em cobrança,ATIVA AJUIZADA,Receita da dívida ativa - Multa - CLT,2010-01-27,SIM,55446.40,2010,S,14,S
3,XXX856.122XX,PF,PRINCIPAL,2142000534105,Benefício Fiscal,ATIVA NAO AJUIZAVEL NEGOCIADA NO SISPAR,R D Ativa - Contribuição Previdenciária Segurados,2020-10-26,NAO,3182.70,2020,N,3,S
4,14.177.984/0001-95,PJ,PRINCIPAL,2160000092445,Em cobrança,ATIVA AJUIZADA,Receita da dívida ativa - COFINS,2000-09-27,SIM,3457828.22,2000,S,3,S
5,02.793.722/0001-76,PJ,PRINCIPAL,2161900910100,Em cobrança,ATIVA EM COBRANCA,R D Ativa - Multa Isolada,2019-02-12,NAO,1521.30,2019,S,1,N


## Criação do campo FAIXA DE VALOR

In [27]:
%%time
# CRIANDO NOVO CAMPO CALCULADO DA FAIXA DE VALOR (https://insightlab.ufc.br/6-truques-do-pandas-para-impulsionar-sua-analise-de-dados/ - VER ITEM 4)

import sys

 
df['VALOR_FAIXA']=pd.cut(

   df['VALOR_CONSOLIDADO'],

   bins=[0, 1000, 5000, 10000, 20000, 50000, 100000, 500000, 1000000, 10000000, sys.maxsize],

   labels=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
)

Wall time: 432 ms


In [28]:
# https://re-thought.com/pandas-value_counts/
print(df["VALOR_FAIXA"].value_counts().sort_index(ascending=True))

1      594826
2     5709103
3     2318277
4     1880276
5     1930306
6     1017555
7     1133213
8      194991
9      191943
10      22430
Name: VALOR_FAIXA, dtype: int64


## Limpando o campo 'RECEITA_PRINCIPAL'

In [29]:
# ANTES DA LIMPEZA DA COLUNA RECEITA_PRINCIPAL
print(df["RECEITA_PRINCIPAL"].value_counts())

DIV ATIVA-SIMPLES NACIONAL                                      2662456
Receita da dívida ativa - IRPF                                  1810681
Receita da dívida ativa - IRPJ                                  1651907
Receita da dívida ativa - COFINS                                1612494
Receita da dívida ativa - CSLL                                  1587306
                                                                 ...   
MULTAS NAO TRIBUTARIAS RFB                                            1
DIV.ATIVA-GAS DIE GLP AL R O P                                        1
Rec. dív. ativa - Medic. adq. dist./varej. - Ret. órg. públ.          1
Rec. dív. ativa - Transp. internac. passag. - Ret. órg. púb.          1
DIV.ATIVA-MULTAS DIREITOS DIFU                                        1
Name: RECEITA_PRINCIPAL, Length: 129, dtype: int64


In [30]:
%%time
# LIMPANDO A COLUNA RECEITA PRINCIPAL (REMOVER TEXTO 'Receita da dívida ativa' e afins)
#https://pt.stackoverflow.com/questions/491195/em-python-como-remover-caracteres-espec%C3%ADficos-de-todos-os-registros-de-apenas-u
df["RECEITA_PRINCIPAL"] = df["RECEITA_PRINCIPAL"].str.replace('Receita da dívida ativa - ', '')
df["RECEITA_PRINCIPAL"] = df["RECEITA_PRINCIPAL"].str.replace(' - Receita da dívida ativa', '')
df["RECEITA_PRINCIPAL"] = df["RECEITA_PRINCIPAL"].str.replace('Rec. dív. ativa - ', '')
df["RECEITA_PRINCIPAL"] = df["RECEITA_PRINCIPAL"].str.replace('Rec. dívida ativa - ', '')
df["RECEITA_PRINCIPAL"] = df["RECEITA_PRINCIPAL"].str.replace('Receita da dív. ativa - ', '')
df["RECEITA_PRINCIPAL"] = df["RECEITA_PRINCIPAL"].str.replace('DIV.ATIVA-', '')
df["RECEITA_PRINCIPAL"] = df["RECEITA_PRINCIPAL"].str.replace('DIV ATIVA-', '')
df["RECEITA_PRINCIPAL"] = df["RECEITA_PRINCIPAL"].str.replace('R D ATIVA - ', '')
df["RECEITA_PRINCIPAL"] = df["RECEITA_PRINCIPAL"].str.replace('R D Ativa - ', '')
df["RECEITA_PRINCIPAL"] = df["RECEITA_PRINCIPAL"].str.replace('R D Ativa -', '')

<timed exec>:5: FutureWarning: The default value of regex will change from True to False in a future version.
<timed exec>:6: FutureWarning: The default value of regex will change from True to False in a future version.
<timed exec>:7: FutureWarning: The default value of regex will change from True to False in a future version.
<timed exec>:8: FutureWarning: The default value of regex will change from True to False in a future version.


Wall time: 1min 23s


In [31]:
# DEPOIS DA LIMPEZA
print(df["RECEITA_PRINCIPAL"].value_counts())

SIMPLES NACIONAL                              2662456
IRPF                                          1810681
IRPJ                                          1651907
COFINS                                        1612494
CSLL                                          1587306
                                               ...   
MULTAS NAO TRIBUTARIAS RFB                          1
GAS DIE GLP AL R O P                                1
Medic. adq. dist./varej. - Ret. órg. públ.          1
Transp. internac. passag. - Ret. órg. púb.          1
MULTAS DIREITOS DIFU                                1
Name: RECEITA_PRINCIPAL, Length: 129, dtype: int64


In [32]:
df.head(100)

,CPF_CNPJ,TIPO_PESSOA,TIPO_DEVEDOR,NUMERO_INSCRICAO,TIPO_SITUACAO_INSCRICAO,SITUACAO_INSCRICAO,RECEITA_PRINCIPAL,DATA_INSCRICAO,INDICADOR_AJUIZADO,VALOR_CONSOLIDADO,ANO_INSCRICAO,INSC_>1_ANO,CONTADOR_NOME,HA_OUTROS_DEBITOS,VALOR_FAIXA
1,11.859.590/0001-29,PJ,PRINCIPAL,2141900331458,Em cobrança,ATIVA EM COBRANCA,SIMPLES NACIONAL,2019-06-18,NAO,55464.40,2019,S,2,S,6
2,05.574.789/0001-08,PJ,PRINCIPAL,2151000016030,Em cobrança,ATIVA AJUIZADA,Multa - CLT,2010-01-27,SIM,55446.40,2010,S,14,S,6
3,XXX856.122XX,PF,PRINCIPAL,2142000534105,Benefício Fiscal,ATIVA NAO AJUIZAVEL NEGOCIADA NO SISPAR,Contribuição Previdenciária Segurados,2020-10-26,NAO,3182.70,2020,N,3,S,2
4,14.177.984/0001-95,PJ,PRINCIPAL,2160000092445,Em cobrança,ATIVA AJUIZADA,COFINS,2000-09-27,SIM,3457828.22,2000,S,3,S,9
5,02.793.722/0001-76,PJ,PRINCIPAL,2161900910100,Em cobrança,ATIVA EM COBRANCA,Multa Isolada,2019-02-12,NAO,1521.30,2019,S,1,N,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,15.070.453/0001-61,PJ,PRINCIPAL,2161500682703,Em cobrança,ATIVA AJUIZADA,COFINS,2015-04-12,SIM,9853.33,2015,S,4,S,3
119,21.456.849/0001-44,PJ,PRINCIPAL,2162100036411,Em cobrança,ATIVA EM COBRANCA,COFINS,2021-03-30,NAO,16976.04,2021,N,23,S,4
120,06.314.820/0001-34,PJ,PRINCIPAL,2121100100074,Em cobrança,ATIVA AJUIZADA,IRPJ,2011-07-26,SIM,3551.05,2011,S,6,S,2
121,XXX520.602XX,PF,PRINCIPAL,2111900981108,Benefício Fiscal,ATIVA NAO AJUIZAVEL NEGOCIADA NO SISPAR,IRPF,2019-08-27,NAO,8769.87,2019,S,9,S,3


## Reduzindo a quantidade de valores do campo 'RECEITA_PRINCIPAL'

In [33]:
# CONTAGEM DE VALORES RECEITA_PRINCIPAL
count_receita_princ = df['RECEITA_PRINCIPAL'].value_counts()
count_receita_princ.head()

SIMPLES NACIONAL    2662456
IRPF                1810681
IRPJ                1651907
COFINS              1612494
CSLL                1587306
Name: RECEITA_PRINCIPAL, dtype: int64

In [34]:
%%time
# Redução de valores do campo 'RECEITA_PRINCIPAL'
# Os dados originais apresentam mais de 100 variedades de tributos, a maioria com baixa frequencia, conforme acima.
# Assim, escolheremos apenas os tributos mais frequentes para manterem seus nomes originais. Os demais serão renomeados para "OUTROS".
def replace(x):
    if x != "SIMPLES NACIONAL" and x != "IRPF" and x != "COFINS" and x != "IRPJ" and x != "CSLL" and x != "PIS" and x != "Multa - CLT" and x != "Multa Isolada" and x != "SIMPLES" and x != "SPU" and x != "IRRF" and x != "SIMP NAC - MEI":
        return 'OUTROS'
    else:
        return str(x)

df['RECEITA_PRINCIPAL'] = df['RECEITA_PRINCIPAL'].apply(replace)

Wall time: 4.85 s


## Criação do campo target PARCELADO

In [35]:
# IDENTIFICANDO VALOR QUE CORRESPONDE AOS DEBITOS PARCELADOS
print(df["TIPO_SITUACAO_INSCRICAO"].value_counts())

Em cobrança                      11476828
Benefício Fiscal                  3452760
Garantia                            39092
Suspenso por decisão judicial       22229
Em negociação                        2013
Name: TIPO_SITUACAO_INSCRICAO, dtype: int64


In [36]:
# AQUI SE OBSERVA QUE O TIPO DE SITUAÇÃO "BENEFÍCIO FISCAL" CONTEM OS DEBITOS QUE FORAM PARCELADOS (PARCELADA, NEGOCIADA NO SISPAR, REFIS, PAES, ETC.). 
df.groupby('TIPO_SITUACAO_INSCRICAO')['SITUACAO_INSCRICAO'].value_counts()

TIPO_SITUACAO_INSCRICAO        SITUACAO_INSCRICAO                                                    
Benefício Fiscal               ATIVA NAO AJUIZAVEL NEGOCIADA NO SISPAR                                   2644713
                               ATIVA AJUIZADA NEGOCIADA NO SISPAR                                         632234
                               ATIVA AJUIZADA PARCELADA LEI 12996/14                                       58913
                               ATIVA AJUIZADA PARC LEI 11941/09 ART 1-DIVIDAS SEM PARCEL. ANTERIOR         36758
                               ATIVA AJUIZADA PARC LEI 11941/09 ART 3-SALDO REMANESCENTE PARCEL            31301
                                                                                                          ...   
Garantia                       ATIVA COM PARCELAMENTO RESCINDIDO E AJUIZ.A SER PROSSEGUIDO - GARANTIA          3
Suspenso por decisão judicial  ATIVA NAO AJUIZAVEL COM EXIGIBILIDADE DO CREDITO SUSPENSA-DEC.JUDICIAL      

In [37]:
%%time
# CRIANDO NOVO CAMPO CALCULADO: PARCELADO (S/N)

import numpy as np

df['PARCELADO'] = np.where(df.TIPO_SITUACAO_INSCRICAO=='Benefício Fiscal', 'S', 'N')

Wall time: 949 ms


In [38]:
# CONFERÊNCIA: OBSERVE QUE A QDE DE SIM CONFERE COM A QDE DE BENEFÍCIO FISCAL (3.452.760)
print(df["PARCELADO"].value_counts())

N    11540162
S     3452760
Name: PARCELADO, dtype: int64


## Procedimentos finais

In [39]:
df.count()

CPF_CNPJ                   14992922
TIPO_PESSOA                14992922
TIPO_DEVEDOR               14992922
NUMERO_INSCRICAO           14992922
TIPO_SITUACAO_INSCRICAO    14992922
SITUACAO_INSCRICAO         14992922
RECEITA_PRINCIPAL          14992922
DATA_INSCRICAO             14992922
INDICADOR_AJUIZADO         14992922
VALOR_CONSOLIDADO          14992922
ANO_INSCRICAO              14992922
INSC_>1_ANO                14992922
CONTADOR_NOME              14992922
HA_OUTROS_DEBITOS          14992922
VALOR_FAIXA                14992920
PARCELADO                  14992922
dtype: int64

In [40]:
df.head(100)

,CPF_CNPJ,TIPO_PESSOA,TIPO_DEVEDOR,NUMERO_INSCRICAO,TIPO_SITUACAO_INSCRICAO,SITUACAO_INSCRICAO,RECEITA_PRINCIPAL,DATA_INSCRICAO,INDICADOR_AJUIZADO,VALOR_CONSOLIDADO,ANO_INSCRICAO,INSC_>1_ANO,CONTADOR_NOME,HA_OUTROS_DEBITOS,VALOR_FAIXA,PARCELADO
1,11.859.590/0001-29,PJ,PRINCIPAL,2141900331458,Em cobrança,ATIVA EM COBRANCA,SIMPLES NACIONAL,2019-06-18,NAO,55464.40,2019,S,2,S,6,N
2,05.574.789/0001-08,PJ,PRINCIPAL,2151000016030,Em cobrança,ATIVA AJUIZADA,Multa - CLT,2010-01-27,SIM,55446.40,2010,S,14,S,6,N
3,XXX856.122XX,PF,PRINCIPAL,2142000534105,Benefício Fiscal,ATIVA NAO AJUIZAVEL NEGOCIADA NO SISPAR,OUTROS,2020-10-26,NAO,3182.70,2020,N,3,S,2,S
4,14.177.984/0001-95,PJ,PRINCIPAL,2160000092445,Em cobrança,ATIVA AJUIZADA,COFINS,2000-09-27,SIM,3457828.22,2000,S,3,S,9,N
5,02.793.722/0001-76,PJ,PRINCIPAL,2161900910100,Em cobrança,ATIVA EM COBRANCA,Multa Isolada,2019-02-12,NAO,1521.30,2019,S,1,N,2,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,15.070.453/0001-61,PJ,PRINCIPAL,2161500682703,Em cobrança,ATIVA AJUIZADA,COFINS,2015-04-12,SIM,9853.33,2015,S,4,S,3,N
119,21.456.849/0001-44,PJ,PRINCIPAL,2162100036411,Em cobrança,ATIVA EM COBRANCA,COFINS,2021-03-30,NAO,16976.04,2021,N,23,S,4,N
120,06.314.820/0001-34,PJ,PRINCIPAL,2121100100074,Em cobrança,ATIVA AJUIZADA,IRPJ,2011-07-26,SIM,3551.05,2011,S,6,S,2,N
121,XXX520.602XX,PF,PRINCIPAL,2111900981108,Benefício Fiscal,ATIVA NAO AJUIZAVEL NEGOCIADA NO SISPAR,IRPF,2019-08-27,NAO,8769.87,2019,S,9,S,3,S


In [41]:
df.tail(100)

,CPF_CNPJ,TIPO_PESSOA,TIPO_DEVEDOR,NUMERO_INSCRICAO,TIPO_SITUACAO_INSCRICAO,SITUACAO_INSCRICAO,RECEITA_PRINCIPAL,DATA_INSCRICAO,INDICADOR_AJUIZADO,VALOR_CONSOLIDADO,ANO_INSCRICAO,INSC_>1_ANO,CONTADOR_NOME,HA_OUTROS_DEBITOS,VALOR_FAIXA,PARCELADO
19651444,03.814.183/0001-77,PJ,PRINCIPAL,1420800000809,Em cobrança,ATIVA AJUIZADA,IRPJ,2008-03-03,SIM,1104510.48,2008,S,8,S,9,N
19651445,03.095.289/0001-68,PJ,PRINCIPAL,1461500207456,Benefício Fiscal,ATIVA AJUIZADA NEGOCIADA NO SISPAR,COFINS,2015-08-12,SIM,6106.63,2015,S,7,S,3,S
19651446,08.304.557/0001-28,PJ,PRINCIPAL,1441600302839,Benefício Fiscal,ATIVA NAO AJUIZAVEL NEGOCIADA NO SISPAR,SIMPLES NACIONAL,2016-02-08,NAO,50371.35,2016,S,1,N,6,S
19651447,09.201.840/0001-97,PJ,PRINCIPAL,1461800149486,Benefício Fiscal,ATIVA NAO AJUIZAVEL NEGOCIADA NO SISPAR,Multa Isolada,2018-03-29,NAO,4215.37,2018,S,1,N,2,S
19651448,26.169.085/0001-20,PJ,PRINCIPAL,1442000219130,Em cobrança,ATIVA A SER AJUIZADA,SIMPLES NACIONAL,2020-06-22,NAO,115062.68,2020,N,10,S,7,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19651571,86.933.363/0001-03,PJ,PRINCIPAL,1459800063130,Benefício Fiscal,ATIVA AJUIZADA NEGOCIADA NO SISPAR,Multa - CLT,1998-03-12,SIM,9487.12,1998,S,27,S,3,S
19651573,15.656.865/0001-88,PJ,PRINCIPAL,1442101139779,Em cobrança,ATIVA EM COBRANCA,SIMP NAC - MEI,2021-12-11,NAO,5173.43,2021,N,1,N,3,N
19651574,06.972.124/0001-15,PJ,PRINCIPAL,1461800011964,Em cobrança,ATIVA AJUIZADA,COFINS,2018-01-26,SIM,109317.16,2018,S,15,S,7,N
19651575,XXX166.661XX,PF,PRINCIPAL,1412100222760,Em cobrança,ATIVA EM COBRANCA,IRPF,2021-02-19,NAO,2372.02,2021,N,6,S,2,N


In [42]:
# EXCLUINDO COLUNAS DESNECESSÁRIAS
df = df.drop(columns=['DATA_INSCRICAO','ANO_INSCRICAO','CONTADOR_NOME', 'TIPO_DEVEDOR'])

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14992922 entries, 1 to 19651576
Data columns (total 12 columns):
 #   Column                   Dtype   
---  ------                   -----   
 0   CPF_CNPJ                 object  
 1   TIPO_PESSOA              object  
 2   NUMERO_INSCRICAO         object  
 3   TIPO_SITUACAO_INSCRICAO  object  
 4   SITUACAO_INSCRICAO       object  
 5   RECEITA_PRINCIPAL        object  
 6   INDICADOR_AJUIZADO       object  
 7   VALOR_CONSOLIDADO        float64 
 8   INSC_>1_ANO              object  
 9   HA_OUTROS_DEBITOS        object  
 10  VALOR_FAIXA              category
 11  PARCELADO                object  
dtypes: category(1), float64(1), object(10)
memory usage: 1.4+ GB


In [44]:
df.head()

,CPF_CNPJ,TIPO_PESSOA,NUMERO_INSCRICAO,TIPO_SITUACAO_INSCRICAO,SITUACAO_INSCRICAO,RECEITA_PRINCIPAL,INDICADOR_AJUIZADO,VALOR_CONSOLIDADO,INSC_>1_ANO,HA_OUTROS_DEBITOS,VALOR_FAIXA,PARCELADO
1,11.859.590/0001-29,PJ,2141900331458,Em cobrança,ATIVA EM COBRANCA,SIMPLES NACIONAL,NAO,55464.40,S,S,6,N
2,05.574.789/0001-08,PJ,2151000016030,Em cobrança,ATIVA AJUIZADA,Multa - CLT,SIM,55446.40,S,S,6,N
3,XXX856.122XX,PF,2142000534105,Benefício Fiscal,ATIVA NAO AJUIZAVEL NEGOCIADA NO SISPAR,OUTROS,NAO,3182.70,N,S,2,S
4,14.177.984/0001-95,PJ,2160000092445,Em cobrança,ATIVA AJUIZADA,COFINS,SIM,3457828.22,S,S,9,N
5,02.793.722/0001-76,PJ,2161900910100,Em cobrança,ATIVA EM COBRANCA,Multa Isolada,NAO,1521.30,S,N,2,N


In [45]:
%%time
# EXPORTANTO DATASET
df.to_csv("05_debitos.csv")

Wall time: 1min 12s
